In [1]:
import pandas as pd
import pyspark
import os
import pyspark
import os
del os.environ['PYSPARK_SUBMIT_ARGS']
from pyspark.sql import SparkSession
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.types import *
import pyspark.sql.functions as F
#spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()

In [2]:
from pyspark.sql.functions import lit
spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()

In [3]:
dict =[{"first":1,"second":3},{"first":1,"second":5},{"first":6,"second":2},{"first":4,"second":4}]

In [4]:
df = pd.DataFrame(dict)

In [5]:
df

,first,second
0,1,3
1,1,5
2,6,2
3,4,4


In [6]:
df.groupby(["first"]).count()

,second
first,
1,2
4,1
6,1


In [7]:
gt_list = [{'from_node':1,'to_node':2},{'from_node':6,'to_node':8},\
           {'from_node':1,'to_node':3},{'from_node':1,'to_node':4},\
           {'from_node':1,'to_node':5},{'from_node':1,'to_node':6},\
           {'from_node':1,'to_node':7},{'from_node':7,'to_node':9},\
           {'from_node':7,'to_node':7},{'from_node':8,'to_node':8},\
           {'from_node':7,'to_node':13},{'from_node':7,'to_node':14},\
           {'from_node':2,'to_node':9},{'from_node':5,'to_node':8},\
           {'from_node':6,'to_node':1},{'from_node':2,'to_node':10},\
           {'from_node':4,'to_node':10},{'from_node':10,'to_node':11},\
           {'from_node':11,'to_node':12},{'from_node':9,'to_node':1},\
           {'from_node':1,'to_node':1}]
schema3 = StructType([
             StructField("from_node", IntegerType(),True),
             StructField("to_node", IntegerType(), True)
        ])
gt_sdf = spark.createDataFrame(gt_list,schema3)

In [8]:
gt_sdf.show()

+---------+-------+
|from_node|to_node|
+---------+-------+
|        1|      2|
|        6|      8|
|        1|      3|
|        1|      4|
|        1|      5|
|        1|      6|
|        1|      7|
|        7|      9|
|        7|      7|
|        8|      8|
|        7|     13|
|        7|     14|
|        2|      9|
|        5|      8|
|        6|      1|
|        2|     10|
|        4|     10|
|       10|     11|
|       11|     12|
|        9|      1|
+---------+-------+
only showing top 20 rows



In [9]:
gt_sdf.createOrReplaceTempView('gt_sdf')
             

In [10]:
frontier_sdf = spark.sql('SELECT to_node FROM gt_sdf')

In [11]:
gt2_sdf = spark.sql('SELECT from_node, MAX(to_node) FROM gt_sdf WHERE from_node > 5 GROUP By from_node ORDER BY MAX(to_node) DESC')
#gt2_sdf = gt2_sdf[gt2_sdf.from_node > 5]


In [12]:
gt2_sdf.show()

+---------+------------+
|from_node|max(to_node)|
+---------+------------+
|        7|          14|
|       11|          12|
|       10|          11|
|        6|           8|
|        8|           8|
|        9|           1|
+---------+------------+



In [13]:
student = [{'Sid':1,'Name':'Jill'},{'Sid':2,'Name':'Bo'},{'Sid':3,'Name':'Maya'}]

In [14]:
takes = [{'Sid':1,'Num':550},{'Sid':1,'Num':520},{'Sid':3,'Num':521},{'Sid':3,'Num':550}]

In [59]:
#gt2_sdf.createOrReplaceTempView('gt2_sdf')
#spark.sql('CREATE INDEX index_name ON gt2_sdf')

In [88]:
schema2 = StructType([
             StructField("Sid", IntegerType(),True),
             StructField("Name", StringType(), True)
        ])
student_sdf = spark.createDataFrame(student,schema2)
student_df = pd.DataFrame.from_dict(student)
schema4 = StructType([
             StructField("Sid", IntegerType(),True),
             StructField("Num", IntegerType(), True)
        ])
takes_sdf = spark.createDataFrame(takes,schema4)
takes_df = pd.DataFrame.from_dict(takes)


In [84]:
student_sdf.show()

+---+----+
|Sid|Name|
+---+----+
|  1|Jill|
|  2|  Bo|
|  3|Maya|
+---+----+



In [85]:
takes_sdf.show()

+---+---+
|Sid|Num|
+---+---+
|  1|550|
|  1|520|
|  3|521|
|  3|550|
+---+---+



In [18]:
takes_sdf.createOrReplaceTempView('takes_sdf')
student_sdf.createOrReplaceTempView('student_sdf')

In [89]:
#output = spark.sql('SELECT * FROM student_sdf as s WHERE EXISTS') #(SELECT * FROM takes_sdf as t WHERE t.SID == s.SID)')
student_sdf.createOrReplaceTempView('student_sdf')
student_sdf = spark.sql('SELECT Sid AS LSid, Name AS LName FROM student_sdf')
student_sdf.createOrReplaceTempView('student_sdf')
#output_sdf = spark.sql('SELECT * FROM student_sdf s FULL JOIN takes_sdf t ON s.LSid == t.Sid WHERE t.Sid IS NULL')
output_sdf = spark.sql('SELECT * FROM student_sdf s INNER JOIN takes_sdf t ON s.LSid == t.Sid')
output_sdf.show()
#output_sdf.createOrReplaceTempView('output_sdf')
#output2_sdf = spark.sql('SELECT LSid AS Sid, LName AS Name FROM output_sdf')
#output_sdf.createOrReplaceTempView('output_sdf')
#output_sdf = spark.sql('SELECT LSid AS Sid, LName AS Name FROM output_sdf')

+----+-----+---+---+
|LSid|LName|Sid|Num|
+----+-----+---+---+
|   1| Jill|  1|550|
|   1| Jill|  1|520|
|   3| Maya|  3|521|
|   3| Maya|  3|550|
+----+-----+---+---+



In [71]:
output2_sdf.show()

+---+----+
|Sid|Name|
+---+----+
|  2|  Bo|
+---+----+



In [58]:
#total_df = student_df.merge(takes_df,left_on='Sid',right_on='Sid',how="outer")
total_df = pd.DataFrame.merge(student_df,takes_df,on='Sid',how='outer')

In [59]:
total_df

,Name,Sid,Num
0,Jill,1,550
1,Jill,1,520
2,Maya,3,521
3,Maya,3,550


In [41]:
test_df = total_df[total_df['Num'].isnull()]

In [42]:
test_df = test_df[['Name','Sid']]

In [43]:
test_df

,Name,Sid
2,Bo,2


In [63]:
total_df.groupby(['Num']).max().reset_index()

,Num,Name,Sid
0,520,Jill,1
1,521,Maya,3
2,550,Maya,3


In [65]:
output2_sdf.createOrReplaceTempView('output2_sdf')

In [68]:
output3_sdf = spark.sql('CREATE INDEX indxr ON output2_sdf WHERE Sid = 2')

ParseException: '\nOperation not allowed: CREATE INDEX(line 1, pos 0)\n\n== SQL ==\nCREATE INDEX indxr ON output2_sdf WHERE Sid = 2\n^^^\n'